In [ ]:
import requests
import pandas as pd
from io import BytesIO
import os
from datetime import datetime, timedelta
import warnings

start_time = datetime.now()

warnings.simplefilter(action='ignore', category=UserWarning)

def download_file(url, username=None, password=None):
    try:
        auth = (username, password) if username and password else None
        response = requests.get(url, auth=auth)
        response.raise_for_status()
        return response.content
    except requests.exceptions.RequestException as e:
        print(f"Ошибка загрузки {url}: {e}")
        return None

def file_to_dataframe(file_content, file_type='excel'):
    if file_content is None:
        return None
    try:
        if file_type == 'csv':
            return pd.read_csv(BytesIO(file_content))
        elif file_type == 'excel':
            df = pd.read_excel(BytesIO(file_content))
            df['Дата регистрации ТС'] = pd.to_datetime(df['Дата регистрации ТС'], dayfirst=True)
            df['Дата регистрации ТС'] = df['Дата регистрации ТС'].dt.strftime('%d.%m.%Y')
            return df
        else:
            print("Неизвестный формат файла")
            return None
    except Exception as e:
        print(f"Ошибка чтения файла: {e}")
        return None

def generate_date_range(start_date, end_date):
    current_date = start_date.replace(day=1)
    date_list = []
    while current_date <= end_date:
        date_list.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=32)
        current_date = current_date.replace(day=1)
    return date_list

def download_and_merge(base_url, locations, start_date, end_date, username, password):
    all_dataframes = []
    date_list = generate_date_range(start_date, end_date)
    
    total_files = len(locations) * len(date_list)
    file_count = 0
    
    for location in locations:
        for date in date_list:
            file_count += 1
            file_url = f"{base_url}/{location}/{location}_{date}.xls"
            print(f"Скачивание {file_count} из {total_files} файлов, {file_url}...")
            
            file_content = download_file(file_url, username, password)
            df = file_to_dataframe(file_content, file_type='excel')
            
            if df is not None:
                df['Дата регистрации ТС'] = pd.to_datetime(df['Дата регистрации ТС'], format='%d.%m.%Y')
                df = df[(df['Дата регистрации ТС'] >= start_date) & (df['Дата регистрации ТС'] <= end_date)]
                df['Дата регистрации ТС'] = df['Дата регистрации ТС'].dt.strftime('%d.%m.%y')
                all_dataframes.append(df)
    
    if all_dataframes:
        df_combined = pd.concat(all_dataframes, ignore_index=True)
        df_combined['Код поставщика'] = df_combined['Код поставщика'].apply(lambda x: str(int(float(x))) if str(x).replace('.', '').isdigit() else str(x))
        df_combined = df_combined[~df_combined['Примечание'].str.contains("банан|арбуз|кгб|россып", case=False, na=False)]
        df_combined = df_combined.rename(columns={'Тип места хранения':'Тип склада МО'})
        df_combined = df_combined[~df_combined['Тип склада МО'].str.contains("КГБ|Вагон|Отобранные задания в ТТ", case=False, na=False)]
        df_combined = df_combined[~df_combined['Вид товара'].str.contains("оборудование|арбуз|Транзит|Паллетный ФТ", case=False, na=False)]

        # Исключаем "Алкоголь" для определенных "Наименование РЦ"
        mask = df_combined['Наименование РЦ'].isin(["РЦ Мурманск", "РЦ Оренбург Ленина"]) & df_combined['Вид товара'].str.contains("Алкоголь", case=False, na=False)
        df_combined = df_combined[~mask]

         # Исключаем строки по столбцу "Поставщик"
        rc_list = [
            "РЦ Шахты", "РЦ Новороссийск Цемдолина", "РЦ Москва", "РЦ Краснодар Индустриальный", "РЦ Тула", "РЦ Иваново", "РЦ Астрахань Тинаки",
            "РЦ Кропоткин", "РЦ Батайск", "РЦ Ерзовка", "РЦ Коломна", "РЦ Смоленск", "РЦ Энгельс", "РЦ Пенза", "РЦ Дзержинск",
            "РЦ Колпино", "РЦ Дмитров", "РЦ Омск", "РЦ Оренбург Ленина", "РЦ Пермь", "РЦ Стерлитамак", "РЦ Тольятти (новый)",
            "РЦ Самара (а)", "РЦ Новосибирск Садовый (новый)", "РЦ Челябинск", "РЦ Сургут", "РЦ Киров", "РЦ Первоуральск",
            "РЦ Зеленодольск", "РЦ Ижевск", "РЦ Тюмень", "РЦ Славянск-на-Кубани", "РЦ Великий Новгород", "РЦ Кемерово",
            "РЦ Ярославль", "РЦ Шушары (а)", "РЦ Мурманск", "РЦ Воронеж", "РЦ Орел (Хардиково)", "РЦ Ногинск (а)",
            "РЦ Тамбов", "РЦ Лермонтов", "РЦ Центр", "РЦ Подольск Федеральный (а)", "РЦ Воронеж оклейка Белгород", 
            "РЦ Казань Свияжск (а)", "РЦ Стерлитамак оклейка"
        ]
        df_combined = df_combined[~df_combined['Поставщик'].isin(rc_list)]
        df_combined = df_combined[~df_combined['Поставщик'].str.contains("ВС ", case=False, na=False)]
        df_combined = df_combined[~df_combined['Поставщик'].str.contains("(СМТС)|(Поставщики НМЦ)|(Поставщик НМЦ)|Не |Незаведенный|Поддоны без документов", case=False, na=False)]

        df_combined['СУММА ДО'] = df_combined['Кол-во поддонов до переборки С'] + df_combined['Кол-во поддонов до переборки Н']
        df_combined['СУММА ПОСЛЕ'] = df_combined['Кол-во паллет после переборки'] / df_combined.groupby('№ Задания приемки')['№ Задания приемки'].transform('count')
        
        df_combined['СУММА ПОСЛЕ'] = df_combined['СУММА ПОСЛЕ'].fillna(0)
        
        df_combined['Сцепка'] = df_combined['Наименование РЦ'].astype(str) + df_combined['Дата регистрации ТС'].astype(str) + df_combined['Номер тс'].astype(str)

        df_combined['Тип ТС'] = df_combined.groupby('Сцепка')['Код поставщика'].transform(lambda x: "Моно" if x.nunique() == 1 else "Сборная")

        df_combined.drop(columns=['Сцепка'], inplace=True)

        return df_combined
    else:
        print("Не удалось загрузить ни один файл")
        return None


username = input("Введите ваш логин: ")
password = input("Введите ваш пароль: ")
start_date = pd.to_datetime(input("Введите дату начала (ГГГГ-ММ-ДД): "))
end_date = pd.to_datetime(input("Введите дату конца (ГГГГ-ММ-ДД): "))


locations_file = r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\Интра\locations.txt"
if os.path.exists(locations_file):
    with open(locations_file, "r", encoding="utf-8") as f:
        locations = [line.strip() for line in f.readlines() if line.strip()]
else:
    print(f"Файл {locations_file} не найден. Используется стандартный список локаций.")
    locations = []

base_url = "http://rc_reports_downloader.intra.corp.tander.ru/reports"

df_combined = download_and_merge(base_url, locations, start_date, end_date, username, password)
if df_combined is not None:
    df_combined.to_excel(r"R:\Лог Условия И Цепи Снабжения\Направление по логистическим контрактам и структуре цепей поставок\10_Файлы для общей работы\ДУЛУ - Аналитики ЛК\1. Справочники\Интра\Интра_.xlsx", index=False, engine='openpyxl')
    end_time = datetime.now()
    execution_time = end_time - start_time
    print(f"Файл ГОТОВ!!! Время выполнения: {execution_time}")